In [4]:
%useLatestDescriptors
%use koog(v = 0.5.0)

In [2]:
import kotlinx.coroutines.runBlocking

val apiKey = System.getenv("OPENAI_API_KEY")
val executor = simpleOpenAIExecutor(apiKey)

runBlocking {
    val res = executor.execute(
        prompt = prompt("prompt") {
            user("hello")
        },
        model = OpenAIModels.Chat.GPT5Mini
    )
    res.last().content
}


Hi — how can I help you today? (I can answer questions, write or edit text, help with code, summarize or translate, brainstorm, and more.)

In [3]:
import ai.koog.agents.features.eventHandler.feature.handleEvents
import ai.koog.agents.features.opentelemetry.feature.OpenTelemetry
import ai.koog.agents.features.opentelemetry.integration.langfuse.addLangfuseExporter
import org.intellij.lang.annotations.Language

val toolRegistry = ToolRegistry {
    tool(AskUser)
}

runBlocking {
    val agent = AIAgent(
        promptExecutor = executor,
        llmModel = OpenAIModels.Chat.GPT5Mini,
        systemPrompt = """
        あなたは旅行プランナーです。ユーザーの指示に従って、旅行計画を立ててください。
        ただしユーザーの指示が少ないときは__ask_user__ツールを使って、1度はユーザーに情報提供を促してください。
        最小限の情報があればそれでよいです。観光スポットなども含めて、あなたが考えて提案してください。
        ユーザーに行きたい場所を尋ねず、あなたが提案してください。
        最小限の情報とは、旅行先、予算のことです。
        あまり細かく聞きすぎず、ある程度分かったところで計画を立ててください
        """.trimIndent(),
        toolRegistry = toolRegistry,
    ) {
        install(OpenTelemetry) {
            setVerbose(true)
            addLangfuseExporter()
        }
        handleEvents {
            onToolCallCompleted {
                when (it.tool) {
                    is AskUser -> {
                        println("---")
                        println("user message: ${it.result}")
                    }

                    else -> {}
                }
            }
        }
    }

    val res = agent.run("2泊3日の札幌旅行の計画を立ててください")
    println("---")
    println("Assistant response:")
    println(res)
}

ご予算を教えてください（お一人当たりの目安で、通貨は日本円でお願いします）。例：3万円、5万円、10万円など。
---
user message: 10月上旬、予算一人5万。二人で行きます。シマエナガをみたいです。
---
Assistant response:
ありがとうございます。いただいた情報（10月上旬、2名、予算お一人5万円、シマエナガを見たい）で、2泊3日の札幌出発（札幌市内滞在中心）プランを作成します。1点確認させてください：お一人5万円のご予算は「ご自宅→札幌の往復（飛行機・新幹線等）を含む総額」ですか、それとも「札幌到着後の現地滞在費（宿泊・食事・現地移動・ツアー等）のみ」ですか？（どちらでも対応しますので教えてください）

以下は、確認前提として「到着は新千歳空港、予算は札幌滞在中を中心に使う／往復交通は別途扱い」と想定した標準プランです。航空券を含めたい場合は回答で教えてください（調整します）。

2泊3日サンプルプラン（シマエナガ観察を中心）

Day1：到着日（札幌市内観光）
- 午前〜午後：新千歳空港到着 → JR快速エアポートで札幌へ（所要約37分）
- チェックインは札幌中心部（大通・すすきの）で便利なホテルを推奨（例：札幌グランドホテル、メルキュール、京王プレリアなど：中級〜やや上の快適クラス）
- 夕方：大通公園、さっぽろテレビ塔、狸小路・すすきので夕食（札幌味噌ラーメン、海鮮、スープカレー等）
- 所要時間：自由。疲れがあれば早めに休息。

Day2：シマエナガ観察の日（早朝発→夕方帰着）
- 早朝発（5:00〜6:00出発が理想）
- オプションA（推奨：日帰りで本気で探す）：
  - JR＋車/ガイドで旭川〜大雪山（大雪山麓、裾合平などの白樺・ダケカンバ林）へ。シマエナガは山地の樹林帯で見られることが多く、早朝がチャンス。
  - 専門のバードウォッチングガイド（半日〜終日）を事前予約すると成功率が高まります（地元ガイドが生息域・ポイントを熟知）。
  - 午後に旭川で軽く観光（旭川ラーメン等）して札幌へ戻る。
- オプションB（移動少なめ・楽に）：
  - 札幌近郊（円山公園・手稲・定山渓周辺の林や郊外フィールド）で早朝バードウォッチング。地元のバードツアー参加を推奨。
- 夕方：札幌で温泉や地元グルメ（海鮮、炉端、ジンギ

In [12]:
USE {
    dependencies {
        val ktorVersion = "3.3.1"
        implementation("io.ktor:ktor-client-core:$ktorVersion")
        implementation("io.ktor:ktor-client-cio:$ktorVersion")
    }
}

In [13]:
import io.ktor.client.HttpClient
import io.ktor.client.call.body
import io.ktor.client.plugins.contentnegotiation.ContentNegotiation
import io.ktor.client.request.get
import io.ktor.client.statement.bodyAsText
import io.ktor.serialization.kotlinx.json.json
import kotlinx.coroutines.runBlocking
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.Json


@kotlinx.serialization.Serializable
data class RouteResponse(
    val routes: List<Route>,
    val code: String,
    val uuid: String
)

@kotlinx.serialization.Serializable
data class Route(
    val duration: Double,
    val distance: Double,
    val geometry: Geometry
)

@Serializable
data class Geometry(
    val coordinates: List<List<Double>>,
)


In [16]:
// Mapbox Directions API用のURL動的構築関数
data class Coordinate(
    val longitude: Double,
    val latitude: Double
)

data class MapboxDirectionsConfig(
    val profile: String = "driving", // driving, walking, cycling, driving-traffic
    val alternatives: Boolean = true,
    val geometries: String = "geojson",
    val language: String = "en",
    val overview: String = "simplified",
    val steps: Boolean = true,
    val accessToken: String
)

fun buildMapboxDirectionsUrl(
    coordinates: List<Coordinate>,
    config: MapboxDirectionsConfig
): String {
    require(coordinates.size >= 2) { "少なくとも2つの座標が必要です" }

    val coordinatesString = coordinates.joinToString(";") { "${it.longitude},${it.latitude}" }

    val queryParams = buildList {
        add("alternatives=${config.alternatives}")
        add("geometries=${config.geometries}")
        add("language=${config.language}")
        add("overview=${config.overview}")
        add("steps=${config.steps}")
        add("access_token=${config.accessToken}")
    }.joinToString("&")

    return "https://api.mapbox.com/directions/v5/mapbox/${config.profile}/$coordinatesString?$queryParams"
}

In [19]:
val client = HttpClient() {
    install(ContentNegotiation) {
        json(Json {
            ignoreUnknownKeys = true
        })
    }
}

val mapboxAccessToken = System.getenv("MAPBOX_ACCESS_TOKEN")
val testCoordinates = listOf(
    Coordinate(-74.027539, 40.801097),
    Coordinate(-74.038602, 40.786767)
)

val config = MapboxDirectionsConfig(accessToken = mapboxAccessToken)

val dynamicUrl = buildMapboxDirectionsUrl(testCoordinates, config)

runBlocking {
    val response =
        client.get("${dynamicUrl}")
    response.body<RouteResponse>()
}


RouteResponse(routes=[Route(duration=137.406, distance=1876.488, geometry=Geometry(coordinates=[[-74.027652, 40.801139], [-74.027878, 40.800786], [-74.027523, 40.800646], [-74.036985, 40.786159]]))], code=Ok, uuid=237ix6D_OfSgFhupL1uKJi-N_cG2FhyeMTCDGoUhueF_3AvJrs9ZXQ==)

## Serialize

In [15]:
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import kotlinx.serialization.json.Json

@Serializable
data class TripPlan(
    val summary: String,
    val step: List<Step>
) {
    @Serializable
    data class Step(
        val date: String,
        val scheduleEntries: List<ScheduleEntry>,
    ) {
        @Serializable
        sealed interface ScheduleEntry {
            val duration: String
            val description: String

            @Serializable
            @SerialName("activity")
            data class Activity(
                override val duration: String,
                override val description: String,
                val location: String,
                val longitude: Double,
                val latitude: Double,
            ) : ScheduleEntry

            @Serializable
            @SerialName("transportation")
            data class Transportation(
                val transportationType: String,
                val from: String,
                val to: String,
                val lineId: String,
                override val duration: String,
                override val description: String
            ) : ScheduleEntry
        }
    }
}

val json = Json { prettyPrint = true }

val output = json.decodeFromString<TripPlan>(
    """
{
  "summary": "鎌倉を巡る日帰り旅行プランです。夫婦でのんびりと鎌倉の魅力を満喫できるよう、定番スポットを中心に、歴史と自然を感じられるコースをご提案します。予算はお一人様2万円、交通手段は電車を利用します。",
  "step": [
    {
      "date": "2025-10-15 08:00",
      "scheduleEntries": [
        {
          "type": "activity",
          "duration": "08:00-09:00",
          "description": "東京駅から鎌倉駅へ移動します。",
          "location": "東京駅",
          "longitude": 139.767125,
          "latitude": 35.681236
        },
        {
          "type": "transportation",
          "transportationType": "電車",
          "from": "東京駅",
          "to": "鎌倉駅",
          "lineId": "1Ug4SirsY6ykxz71Ox_IeAVWVgOMVINOIMEU1sJiiTVf3G0qMLvPqQ==",
          "duration": "約1時間",
          "description": "JR横須賀線を利用します。座席を確保して、快適な移動をお楽しみください。SuicaやPASMOなどのICカードを利用すると便利です。料金は1000円程度です。"
        },
        {
          "type": "activity",
          "duration": "09:00-11:00",
          "description": "鎌倉のシンボル、鶴岡八幡宮を参拝します。源頼朝ゆかりの神社で、広大な境内には見どころがたくさんあります。",
          "location": "鶴岡八幡宮",
          "longitude": 139.553611,
          "latitude": 35.329444
        },
        {
          "type": "transportation",
          "transportationType": "徒歩",
          "from": "鎌倉駅",
          "to": "鶴岡八幡宮",
          "lineId": "nvfQWRv7cXAtjTVdqRQteQ3NhHUUSoDDM88bBrrqdNUHdYoZ1GisVA==",
          "duration": "15分",
          "description": "駅から徒歩でアクセスできます。参道を歩きながら、鎌倉の街並みを楽しみましょう。"
        },
        {
          "type": "activity",
          "duration": "11:30-12:30",
          "description": "鎌倉駅周辺でお昼ご飯。",
          "location": "鎌倉駅周辺の飲食店",
          "longitude": 139.553611,
          "latitude": 35.329444
        }
      ]
    },
    {
      "date": "2025-10-15 13:00",
      "scheduleEntries": [
        {
          "type": "transportation",
          "transportationType": "徒歩",
          "from": "鎌倉駅",
          "to": "大仏",
          "lineId": "HKYIBHb1cENf0mcUSehMBAmHk_RbGoCiSB0fMEBP6jvZUfTCQlVyig==",
          "duration": "15分",
          "description": "江ノ電に乗って長谷駅へ。そこから徒歩で高徳院へ向かいます。"
        },
        {
          "type": "activity",
          "duration": "13:00-15:00",
          "description": "鎌倉大仏を見学します。高さ約13メートルの巨大な仏像は圧巻です。胎内に入ることもできます。",
          "location": "鎌倉大仏（高徳院）",
          "longitude": 139.547222,
          "latitude": 35.328333
        }
      ]
    },
    {
      "date": "2025-10-15 16:00",
      "scheduleEntries": [
        {
          "type": "transportation",
          "transportationType": "電車",
          "from": "鎌倉駅",
          "to": "東京駅",
          "lineId": "5oSsBzf4XrBeU0-ee2HgmzwSNqaKUb7WnYLN2gkrH8xOuKUTuqMFgg==",
          "duration": "約1時間",
          "description": "鎌倉駅からJR横須賀線で東京駅へ戻ります。"
        },
        {
          "type": "activity",
          "duration": "17:00",
          "description": "東京駅に到着。",
          "location": "東京駅",
          "longitude": 139.767125,
          "latitude": 35.681236
        }
      ]
    }
  ]
}
"""
)
output

TripPlan(summary=鎌倉を巡る日帰り旅行プランです。夫婦でのんびりと鎌倉の魅力を満喫できるよう、定番スポットを中心に、歴史と自然を感じられるコースをご提案します。予算はお一人様2万円、交通手段は電車を利用します。, step=[Step(date=2025-10-15 08:00, scheduleEntries=[Activity(duration=08:00-09:00, description=東京駅から鎌倉駅へ移動します。, location=東京駅, longitude=139.767125, latitude=35.681236), Transportation(transportationType=電車, from=東京駅, to=鎌倉駅, lineId=1Ug4SirsY6ykxz71Ox_IeAVWVgOMVINOIMEU1sJiiTVf3G0qMLvPqQ==, duration=約1時間, description=JR横須賀線を利用します。座席を確保して、快適な移動をお楽しみください。SuicaやPASMOなどのICカードを利用すると便利です。料金は1000円程度です。), Activity(duration=09:00-11:00, description=鎌倉のシンボル、鶴岡八幡宮を参拝します。源頼朝ゆかりの神社で、広大な境内には見どころがたくさんあります。, location=鶴岡八幡宮, longitude=139.553611, latitude=35.329444), Transportation(transportationType=徒歩, from=鎌倉駅, to=鶴岡八幡宮, lineId=nvfQWRv7cXAtjTVdqRQteQ3NhHUUSoDDM88bBrrqdNUHdYoZ1GisVA==, duration=15分, description=駅から徒歩でアクセスできます。参道を歩きながら、鎌倉の街並みを楽しみましょう。), Activity(duration=11:30-12:30, description=鎌倉駅周辺でお昼ご飯。, location=鎌倉駅周辺の飲食店, longitude=139.553611, latitude=35.329444)]), Step(date=2025-10-1